<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Определение-текущего-уровня-потребительской-лояльности-(NPS)-в-телекоммуникационной-кампании" data-toc-modified-id="Определение-текущего-уровня-потребительской-лояльности-(NPS)-в-телекоммуникационной-кампании-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Определение текущего уровня потребительской лояльности (NPS) в телекоммуникационной кампании</a></span><ul class="toc-item"><li><span><a href="#Подготовка-данных-для-дашборда" data-toc-modified-id="Подготовка-данных-для-дашборда-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Подготовка данных для дашборда</a></span></li></ul></li><li><span><a href="#Ссылка-на-дашборд" data-toc-modified-id="Ссылка-на-дашборд-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Ссылка на дашборд</a></span></li><li><span><a href="#Ссылка-на-презентацию" data-toc-modified-id="Ссылка-на-презентацию-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Ссылка на презентацию</a></span></li></ul></div>

# Определение текущего уровня потребительской лояльности (NPS) в телекоммуникационной кампании

**Заказчик исследования**: Телекоммуникационная кампания 

**Цель исследования:** Определить текущий уровень потребительской лояльности (NPS) в кампании 

**Объект исследования:** Результаты опроса потребителей кампании, хранящиеся в БД SQLlite  `telecomm_csi.db`

**Задачи, которые будут решены в ходе исследования:**

- Подготовка данных для дашборда;
- Создание дашборда в Tableau, который позволит получить ответы на следующие вопросы:
    - Как распределены участники опроса по возрасту, полу и возрасту? Каких пользователей больше: новых или старых? Пользователи из каких городов активнее участвовали в опросе?
    - Какие группы пользователей наиболее лояльны к сервису? Какие менее?
    - Какой общий NPS среди всех опрошенных?
    - Как можно описать клиентов, которые относятся к группе cторонников (англ. promoters)?
- Публикация дашборда на сайте Tableau Public;
- Оформление итоговой презентации.




## Подготовка данных для дашборда

In [1]:
# импорт нужных библиотек 
import pandas as pd 
from sqlalchemy import create_engine

In [2]:
# создаем соединение к БД SQLite
path_to_db = '***.db'
engine = create_engine(f'sqlite:///{path_to_db}', echo = False)

Перед подготовкой витрины данных для дашборда посмотрим содержимое таблиц БД 

In [3]:
# выведем первые 5 записей таблицы user
query_user = '''
    SELECT * FROM user;
'''

df_user = pd.read_sql(query_user, engine)
display(df_user.head())
df_user.info()
display(df_user.loc[(df_user['age'].isna()) | (df_user['gender_segment'].isna())])

,user_id,lt_day,age,gender_segment,os_name,cpe_type_name,location_id,age_gr_id,tr_gr_id,lt_gr_id,nps_score
0,A001A2,2320,45.0,1.0,ANDROID,SMARTPHONE,55,5,5,8,10
1,A001WF,2344,53.0,0.0,ANDROID,SMARTPHONE,21,5,5,8,10
2,A003Q7,467,57.0,0.0,ANDROID,SMARTPHONE,28,6,9,6,10
3,A004TB,4190,44.0,1.0,IOS,SMARTPHONE,38,4,4,8,10
4,A004XT,1163,24.0,0.0,ANDROID,SMARTPHONE,39,2,6,8,10


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502493 entries, 0 to 502492
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   user_id         502493 non-null  object 
 1   lt_day          502493 non-null  int64  
 2   age             501939 non-null  float64
 3   gender_segment  501192 non-null  float64
 4   os_name         502493 non-null  object 
 5   cpe_type_name   502493 non-null  object 
 6   location_id     502493 non-null  int64  
 7   age_gr_id       502493 non-null  int64  
 8   tr_gr_id        502493 non-null  int64  
 9   lt_gr_id        502493 non-null  int64  
 10  nps_score       502493 non-null  int64  
dtypes: float64(2), int64(6), object(3)
memory usage: 42.2+ MB


,user_id,lt_day,age,gender_segment,os_name,cpe_type_name,location_id,age_gr_id,tr_gr_id,lt_gr_id,nps_score
751,A1E59W,4055,NaN,NaN,ANDROID,SMARTPHONE,13,8,5,8,1
950,A1S6VL,335,40.0,NaN,ANDROID,TABLET,14,4,8,5,8
1209,A28ZDT,4243,NaN,NaN,ANDROID,SMARTPHONE,56,8,5,8,5
1321,A2GLPQ,2354,NaN,NaN,ANDROID,SMARTPHONE,16,8,7,8,1
2113,A3XMNW,41,32.0,NaN,ANDROID,SMARTPHONE,41,3,4,2,1
...,...,...,...,...,...,...,...,...,...,...,...
500834,ZWQJRW,220,NaN,0.0,ANDROID,SMARTPHONE,40,8,4,5,1
500950,ZWY9NM,49,35.0,NaN,ANDROID,SMARTPHONE,27,4,5,2,10
501246,ZXK9AK,3550,NaN,NaN,ANDROID,SMARTPHONE,50,8,5,8,4
502376,ZZRS2G,345,NaN,0.0,ANDROID,SMARTPHONE,28,8,5,5,5


In [4]:
# выведем первые 5 записей таблицы location
query_location = '''
    SELECT * FROM location;
'''

df_location = pd.read_sql(query_location, engine)
df_location.head()

,location_id,city,country
0,1,Архангельск,Россия
1,2,Астрахань,Россия
2,3,Балашиха,Россия
3,4,Барнаул,Россия
4,5,Белгород,Россия


In [5]:
# выведем первые 10 записей таблицы age_segment
query_age_segment = '''
    SELECT * FROM age_segment;
'''

df_age_segment = pd.read_sql(query_age_segment, engine)
df_age_segment.head(10)

,age_gr_id,bucket_min,bucket_max,title
0,1,NaN,15.0,01 до 16
1,2,16.0,24.0,02 16-24
2,3,25.0,34.0,03 25-34
3,4,35.0,44.0,04 35-44
4,5,45.0,54.0,05 45-54
5,6,55.0,64.0,06 55-64
6,7,66.0,NaN,07 66 +
7,8,NaN,NaN,08 n/a


In [6]:
# выведем первые 5 записей таблицы traffic_segment
query_traffic_segment = '''
    SELECT * FROM traffic_segment;
'''

df_traffic_segment = pd.read_sql(query_traffic_segment, engine)
print(df_traffic_segment.head())
print(df_traffic_segment.shape[0])

   tr_gr_id  bucket_min  bucket_max        title
0         1        0.00        0.00         01 0
1         2        0.00        0.01    01 0-0.01
2         3        0.01        0.10  02 0.01-0.1
3         4        0.10        1.00     03 0.1-1
4         5        1.00        5.00       04 1-5
25


In [7]:
# выведем первые 10 записей таблицы lifetime_segment
query_lifetime_segment = '''
    SELECT * FROM lifetime_segment;
'''

df_lifetime_segment = pd.read_sql(query_lifetime_segment, engine)
df_lifetime_segment.head(10)

,lt_gr_id,bucket_min,bucket_max,title
0,1,1.0,1.0,01 1
1,2,2.0,2.0,02 2
2,3,3.0,3.0,03 3
3,4,4.0,6.0,04 4-6
4,5,7.0,12.0,05 7-12
5,6,13.0,24.0,06 13-24
6,7,25.0,36.0,07 25-36
7,8,36.0,NaN,08 36+


Напишем запрос для выгрузки следующих поля для витрины, которая будет использована для построения дашборда:

- `user_id`	- Идентификатор клиента
- `lt_day`	- Количество дней «жизни» клиента 
- `is_new`	- Поле хранит информацию о том, является ли клиент новым (1 - новый, 0 - нет). Клиента считают новым, если количество дней «жизни» составляет не более 365 дней. 
- `age`	- Возраст
- `gender_segment`	- Пол (Ж - женский, М - мужской)
- `os_name`	- Тип операционной системы
- `cpe_type_name`	- Тип устройства
- `country`	- Страна проживания
- `city`	- Город проживания
- `age_segment`	- Возрастной сегмент
- `traffic_segment`	- Сегмент по объёму потребляемого трафика
- `lifetime_segment`	- Сегмент по количеству дней «жизни»
- `nps_score`	- Оценка клиента в NPS-опросе
- `nps_group`	- Поле хранит информацию о том, к какой группе относится оценка клиента в опросе

Значения для полей `age_segment`, `traffic_segment` и `lifetime_segment` будут браться из полей `title` соответсвующих таблиц. Как видим из выгрузок данных из таблиц, значения `title` в исходных таблицах содержатся в формате "<порядковый номер> <название>". При подгтовке данных для витрины оставим в полях только название 

In [9]:
query_view_dash = '''
SELECT u.user_id, 
       u.lt_day,
       CASE 
	       WHEN u.lt_day < 366 THEN '1'
	       ELSE '0'
       END AS is_new,
       u.age,
       CASE 
	       WHEN u.gender_segment = 1 THEN 'Ж'
	       WHEN u.gender_segment = 0 THEN 'М'
       END AS gender_segment,
       u.os_name,
       u.cpe_type_name,
       l.country,
       l.city,
       SUBSTRING(a_seg.title, INSTR(a_seg.title, ' '), LENGTH(a_seg.title)) AS age_segment,
       SUBSTRING(ts.title, INSTR(ts.title, ' '), LENGTH(ts.title)) AS traffic_segment, 
       SUBSTRING(ls.title, INSTR(ls.title, ' '), LENGTH(ls.title)) AS lifetime_segment, 
       u.nps_score,
       CASE 
	       WHEN u.nps_score BETWEEN 9 AND 10 THEN 'Сторонники'
	       WHEN u.nps_score BETWEEN 7 AND 8 THEN 'Нейтралы'
	       WHEN u.nps_score BETWEEN 0 AND 6 THEN 'Критики'
       END AS nps_group
       
FROM user AS u
LEFT JOIN location AS l ON u.location_id = l.location_id 
LEFT JOIN age_segment AS a_seg ON u.age_gr_id = a_seg.age_gr_id 
LEFT JOIN traffic_segment AS ts ON u.tr_gr_id = ts.tr_gr_id
LEFT JOIN lifetime_segment AS ls ON u.lt_gr_id = ls.lt_gr_id
'''

df_dash = pd.read_sql(query_view_dash, engine)

In [10]:
# получим общую информацию о датасете 
df_dash.info()
display(df_dash.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502493 entries, 0 to 502492
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   user_id           502493 non-null  object 
 1   lt_day            502493 non-null  int64  
 2   is_new            502493 non-null  object 
 3   age               501939 non-null  float64
 4   gender_segment    501192 non-null  object 
 5   os_name           502493 non-null  object 
 6   cpe_type_name     502493 non-null  object 
 7   country           502493 non-null  object 
 8   city              502493 non-null  object 
 9   age_segment       502493 non-null  object 
 10  traffic_segment   502493 non-null  object 
 11  lifetime_segment  502493 non-null  object 
 12  nps_score         502493 non-null  int64  
 13  nps_group         502493 non-null  object 
dtypes: float64(1), int64(2), object(11)
memory usage: 53.7+ MB


,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,0,45.0,Ж,ANDROID,SMARTPHONE,Россия,Уфа,45-54,1-5,36+,10,Сторонники
1,A001WF,2344,0,53.0,М,ANDROID,SMARTPHONE,Россия,Киров,45-54,1-5,36+,10,Сторонники
2,A003Q7,467,0,57.0,М,ANDROID,SMARTPHONE,Россия,Москва,55-64,20-25,13-24,10,Сторонники
3,A004TB,4190,0,44.0,Ж,IOS,SMARTPHONE,Россия,РостовнаДону,35-44,0.1-1,36+,10,Сторонники
4,A004XT,1163,0,24.0,М,ANDROID,SMARTPHONE,Россия,Рязань,16-24,5-10,36+,10,Сторонники


Подготовленная витрина с данными содержит 502 493 строк. Пропусков в данных нет. Типы данных соответсвуют содержащихся в них значениях. Условия предобразования данных выполнены корректно. Таблица применима для построения дашборда и может быть выгружена в `.csv` файл.

In [11]:
# Получившуюся таблицу сохраним как `.csv` файл.
df_dash.to_csv('user_nps_dash.csv', index = False)

# Ссылка на дашборд


https://public.tableau.com/app/profile/natalia7606/viz/Project2_16757061423070/DashNPS?publish=yes 

# Ссылка на презентацию


https://disk.yandex.ru/d/NdCOhQ-2ei0E_Q 
    